# Azure Learning - App Service

In [1]:
az login

[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "415d05c4-204b-4df3-b7db-fb5faa4abb73",
    "id": "586f41cd-533d-41bd-9be4-88f5657a9ed4",
    "isDefault": false,
    "managedByTenants": [],
    "name": "Azure Cloud POC",
    "state": "Enabled",
    "tenantId": "415d05c4-204b-4df3-b7db-fb5faa4abb73",
    "user": {
      "name": "Randall.Thornton@cc-racks.com",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "415d05c4-204b-4df3-b7db-fb5faa4abb73",
    "id": "139ab8b0-8b6c-4574-ba4e-cb7f38419e03",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Visual Studio Professional Subscription",
    "state": "Enabled",
    "tenantId": "415d05c4-204b-4df3-b7db-fb5faa4abb73",
    "user": {
      "name": "Randall.Thornton@cc-racks.com",
      "type": "user"
    }
  }
]


In [2]:
# Set the subscription

az account set --subscription '139ab8b0-8b6c-4574-ba4e-cb7f38419e03'

In [5]:
# Create the resource group
$resourceGroupName = "az-learn-group"
$appServicePlanName = "appServicePlan"
$webAppName = "az-learn-web-app-89"

az group create --name "$resourceGroupName" --location eastus

{
  "id": "/subscriptions/139ab8b0-8b6c-4574-ba4e-cb7f38419e03/resourceGroups/az-learn-group",
  "location": "eastus",
  "managedBy": null,
  "name": "az-learn-group",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}


In [6]:
# Create the infrastructure

az deployment group create --resource-group "$resourceGroupName" --template-file .\main.bicep --parameters appServicePlanName=$appServicePlanName webAppName=$webAppName

{
  "id": "/subscriptions/139ab8b0-8b6c-4574-ba4e-cb7f38419e03/resourceGroups/az-learn-group/providers/Microsoft.Resources/deployments/main",
  "location": null,
  "name": "main",
  "properties": {
    "correlationId": "21b87548-931e-4567-9daf-ee1288fc5abb",
    "debugSetting": null,
    "dependencies": [
      {
        "dependsOn": [
          {
            "id": "/subscriptions/139ab8b0-8b6c-4574-ba4e-cb7f38419e03/resourceGroups/az-learn-group/providers/Microsoft.Web/serverfarms/appServicePlan",
            "resourceGroup": "az-learn-group",
            "resourceName": "appServicePlan",
            "resourceType": "Microsoft.Web/serverfarms"
          }
        ],
        "id": "/subscriptions/139ab8b0-8b6c-4574-ba4e-cb7f38419e03/resourceGroups/az-learn-group/providers/Microsoft.Web/sites/az-learn-web-app-89",
        "resourceGroup": "az-learn-group",
        "resourceName": "az-learn-web-app-89",
        "resourceType": "Microsoft.Web/sites"
      },
      {
        "dependsOn": [

In [18]:
# Build the App

dotnet publish .\app-service.csproj -c Release -o .\publish --self-containerd true

Compress-Archive -Path .\publish\* -DestinationPath .\app.zip -Update

MSBuild version 17.9.4+90725d08d for .NET
MSBUILD : error MSB1001: Unknown switch.
    Full command line: 'C:\Program Files\dotnet\sdk\8.0.200\MSBuild.dll -maxcpucount -verbosity:m -restore -target:Publish --property:_IsPublishing=true -property:PublishDir=C:\Users\randa\source\repos\azure-learning\app-service\publish -property:_CommandLineDefinedOutputPath=true -property:Configuration=Release -property:DOTNET_CLI_DISABLE_PUBLISH_AND_PACK_RELEASE=true .\app-service.csproj --self-containerd true -distributedlogger:Microsoft.DotNet.Tools.MSBuild.MSBuildLogger,C:\Program Files\dotnet\sdk\8.0.200\dotnet.dll*Microsoft.DotNet.Tools.MSBuild.MSBuildForwardingLogger,C:\Program Files\dotnet\sdk\8.0.200\dotnet.dll'
  Switches appended by response files:
Switch: --self-containerd

For switch syntax, type "MSBuild -help"


Error: Command failed: SubmitCode: dotnet publish .\app-service.csproj -c Release -o  ...

In [19]:
# Deploy to App Zip

az webapp deployment source config-zip --resource-group "$resourceGroupName" --name "$webAppName" --src .\app.zip

{
  "active": true,
  "author": "N/A",
  "author_email": "N/A",
  "complete": true,
  "deployer": "ZipDeploy",
  "end_time": "2024-05-12T18:33:23.463315Z",
  "id": "af2fb5c1c9354675aa53b80f878b1c09",
  "is_readonly": true,
  "is_temp": false,
  "last_success_end_time": "2024-05-12T18:33:23.463315Z",
  "log_url": "https://az-learn-web-app-89.scm.azurewebsites.net/api/deployments/latest/log",
  "message": "Created via a push deployment",
  "progress": "",
  "provisioningState": "Succeeded",
  "received_time": "2024-05-12T18:33:22.5101882Z",
  "site_name": "az-learn-web-app-89",
  "start_time": "2024-05-12T18:33:22.5883104Z",
  "status": 4,
  "status_text": "",
  "url": "https://az-learn-web-app-89.scm.azurewebsites.net/api/deployments/latest"
}


In [ ]:
# Cleanup the resources

az group delete --name "$resourceGroupName" --yes --no-wait